In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys


In [ ]:
# Allow imports from parent directory 
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im/35273613#35273613
#module_path = os.path.abspath(os.path.join(os.pardir))
module_path = 'C:\\Users\\agomez\\Dropbox\\Harvard\\LittleProjects\\StochasticPS\\programs\\'
if module_path not in sys.path:
    sys.path.append(module_path)
sys.path

In [ ]:
import time
import datetime

import pandas as pd
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

import seaborn as sns; sns.set(style="ticks", color_codes=True)

import itertools
import collections
import warnings
import IPython.display
import scipy.stats
import networkx as nx
from operator import itemgetter

plt.style.use('seaborn-white')
plt.rc('font', family='serif', serif='Helvetica')
plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('axes', labelsize=16)


from sklearn.preprocessing import StandardScaler, RobustScaler, FunctionTransformer, PolynomialFeatures
from sklearn.decomposition import PCA, NMF, FactorAnalysis
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

from numpy.random import exponential, negative_binomial, randint, choice, binomial
from random import shuffle

import statsmodels.formula.api as smf



In [ ]:
import EComm_0001_complexities 

# Paths
path_fig = 'C:\\Users\\agomez\\Dropbox\\Harvard\\LittleProjects\\StochasticPS\\figures\\'
path_data = 'C:\\Users\\agomez\\Dropbox\\Harvard\\LittleProjects\\StochasticPS\\data\\'
path_outputdata = 'C:\\Users\\agomez\\Dropbox\\Harvard\\LittleProjects\\StochasticPS\\outputdata\\'
path_inputdata = 'C:\\Users\\agomez\\Dropbox\\Harvard\\LittleProjects\\StochasticPS\\inputdata\\'

# format of figures
figformat = "pdf"
save2file = False

# Loading SITC 2015 data 
from: https://intl-atlas-downloads.s3.amazonaws.com/index.html

In [ ]:
#ccpy_filepath = "https://intl-atlas-downloads.s3.amazonaws.com/CCPY/S2_final_{yr}.dta"
ccpy_filepath = path_inputdata + "S2_final_{yr}.csv"

#cpy_filepath = "https://intl-atlas-downloads.s3.amazonaws.com/CPY/S2_final_cpy_all.dta"
cpy_filepath = path_inputdata + "S2_final_cpy_all.dta"

#ctyregions_filepath = "https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv"
ctyregions_filepath = path_data + "all.csv"

gdp_filepath = path_data + "WorldBank_GDPperCapita_1962_to_2015.json"

In [ ]:
rowvarstring = 'exporter'
colvarstring = 'commoditycode'
valvarstring = 'mcp'
year = 2015

In [ ]:
ctyregs_df = pd.read_csv(ctyregions_filepath)
ctyregs_df.head()

In [ ]:
ctyregs_df = ctyregs_df[~ctyregs_df['sub-region'].isnull()]
ctyregs_df.shape

In [ ]:
###### LOADING THE DATA #######
longdf = pd.read_stata(cpy_filepath)

In [ ]:
longdf.dtypes

In [ ]:
print(longdf.shape)
longdf.head()

In [ ]:
# subsetting to the year
onlydecades_df = longdf[longdf['year'].astype(str).astype(int)%5 == 0] 
longdf = longdf[longdf['year'].astype(str).astype(int)==year]
print(onlydecades_df.shape, longdf.shape)

### Getting GDP data

In [ ]:
###### LOADING THE DATA #######
gdp_longdf = pd.read_json(gdp_filepath)
gdp_longdf = gdp_longdf.sort_values(['iso3c', 'year'])

In [ ]:
gdp_longdf = gdp_longdf[gdp_longdf['year'].astype(str).astype(int)%5 == 0]
gdp_longdf = gdp_longdf[~gdp_longdf['gdp_per_capita_constant2010USD'].isnull()]

In [ ]:
gdp_longdf

In [ ]:
gdp_longdf.tail(4)

## Getting the codes and checking consistency

In [ ]:
exp_codes = np.sort(longdf['exporter'].unique())
prod_codes = np.sort(longdf['commoditycode'].unique())
gdpcty_codes = np.sort(gdp_longdf['iso3c'].unique())

In [ ]:
iso_codes = np.sort(ctyregs_df['alpha-3'].unique())

In [ ]:
# the country codes that are in our dataset for which we do not have other meta-info
list(set(exp_codes) - set(iso_codes))

In [ ]:
cty_codes = np.array(list(set(exp_codes) & set(iso_codes) & set(gdpcty_codes)))
len(cty_codes)

In [ ]:
longdf = longdf[longdf[rowvarstring].isin(cty_codes)]

In [ ]:
longdf.shape

### using the country codes to filder the data of GDP

In [ ]:
gdp_longdf = gdp_longdf[gdp_longdf['iso3c'].isin(cty_codes)][['iso3c', 'country', 'year', 
                                                              'gdp_per_capita_PPP_constant_2011_international_dollar', 
                                                              'gdp_per_capita_PPP_current_international_dollar', 
                                                              'gdp_per_capita_constant2010USD']]
print((np.unique(gdp_longdf.iso3c), len(np.unique(gdp_longdf.iso3c))))

## Converting long to wide

In [ ]:
# ---------------------
# From long to wide format
Mcp_widedf = longdf[longdf['year'].astype(str).astype(int)==year][[rowvarstring, colvarstring, valvarstring]].pivot(index=rowvarstring, 
                                                                       columns=colvarstring, 
                                                                       values=valvarstring).fillna(0.0)

In [ ]:
Mcp_widedf.shape

In [ ]:
Mcp_widedf.head()

In [ ]:
nmat, ncP, npP = EComm_0001_complexities.ReorderingMatrix(Mcp_widedf)
plt.spy(nmat, aspect='auto')
plt.show()

In [ ]:
divty = Mcp_widedf.sum(axis=1)
ubity = Mcp_widedf.sum(axis=0)
print(np.sum(divty==0), np.sum(ubity==0))
print(np.sum(divty==1), np.sum(ubity==1))


In [ ]:
# Calculating the c2c and p2p matrices, eigenvalues and left-eigenvectors
#(Mc2c, Dc, Vc), (Mp2p, Dp, Vp) = EComm_0001_complexities.ECeigenvecs(Mcp_widedf)
(Mc2c, Dc, leftVc, rightVc) = EComm_0001_complexities.ECeigenvecs(Mcp_widedf)
minsize = min(Mcp_widedf.shape)
print(minsize)

# Calculating the right-eigenvectors
#rightDc, rightVc = np.linalg.eig(Mc2c)

In [ ]:
fig = plt.figure(figsize=(20,5))

ax1 = fig.add_subplot(1,3,1)
ax1.hist(Dc.real[:minsize])
#ax1.set_xscale('log')
ax1.set_yscale('log')

ax2 = fig.add_subplot(1,3,2)
ax2.hist(Dc.imag[:minsize])
#ax2.set_xscale('log')
ax2.set_yscale('log')

ax3 = fig.add_subplot(1,3,3)
ax3.scatter(Dc.real[10:minsize], Dc.imag[10:minsize])
#ax3.set_xscale('log')
#ax3.set_yscale('log')

plt.show()

In [ ]:
fig = plt.figure(figsize=(16,5))
plt.subplots_adjust(wspace=0.2)

ax1 = fig.add_subplot(1,2,1)
# histogram of country eigenvalues
sns.distplot(Dc.real, bins=50, rug=True, kde=False, norm_hist=True, ax=ax1, axlabel="Eigen value")
sns.kdeplot(Dc.real, bw=.005, ax=ax1)

ax2 = fig.add_subplot(1,2,2)
# histogram of country ECI's
sns.distplot(leftVc[:,1], bins=40, rug=True, kde=False, norm_hist=True, ax=ax2, axlabel="ECI")
sns.kdeplot(leftVc[:,1], bw=.01, ax=ax2)
plt.show()

In [ ]:
# Counting the number of eigenvalues larger than 0.3~0.4 is a good measure of the number of communities
np.sum(Dc>0.2)

### Which communities to use?

In [ ]:
communitycolumn = 'region'

# left-eigenvalue data frame
leftVc_df = pd.DataFrame(leftVc, index=Mcp_widedf.index)
leftVc_df['Community'] = [ctyregs_df[ctyregs_df['alpha-3']==cname][communitycolumn].values[0] for cname in Mcp_widedf.index.values]

# right-eigenvalue data frame
rightVc_df = pd.DataFrame(rightVc, index=Mcp_widedf.index)
rightVc_df['Community'] = [ctyregs_df[ctyregs_df['alpha-3']==cname][communitycolumn].values[0] for cname in Mcp_widedf.index.values]

print(leftVc_df.head())
print(rightVc_df.head())

In [ ]:
leftVc_df.head()

In [ ]:
realcomms = np.unique(leftVc_df['Community'].values)
kcomm = len(realcomms)
mycolors = sns.color_palette("Set1", n_colors=kcomm, desat=.5)

communities_vec = realcomms
numcommunities = len(communities_vec)
rncomm = np.arange(numcommunities)
mycolors = sns.color_palette("Set1", n_colors=numcommunities, desat=.5)

(communities_vec, numcommunities, rncomm)


In [ ]:
cty_marker_sizes = (Mcp_widedf.sum(axis=1).values/(0.8*np.min(Mcp_widedf.sum(axis=1).values)))**1.1
(np.min(cty_marker_sizes), np.max(cty_marker_sizes))

In [ ]:
fig = plt.figure(figsize=(15,10))

#ax1 = fig.add_subplot(131)
#ax2 = fig.add_subplot(132)
#ax3 = fig.add_subplot(133)


#########################################################################
# FIRST ROW: LEFT-EIGENVECTORS
ax1 = fig.add_subplot(2,3,1)
for color, i, target_name in zip(mycolors, rncomm, np.array(communities_vec)):
    ax1.scatter(leftVc_df[leftVc_df.Community==target_name][1], leftVc_df[leftVc_df.Community==target_name][2],
                c=color, label=target_name, 
                s=cty_marker_sizes[leftVc_df.Community==target_name])
ax1.set_xlabel('2nd left-eigenvector (aka, ECI)', fontsize=16)
ax1.set_ylabel('3rd left-eigenvector', fontsize=16)
#plt.legend(loc="best", shadow=False, scatterpoints=1)

ax2 = fig.add_subplot(2,3,2)
for color, i, target_name in zip(mycolors, rncomm, np.array(communities_vec)):
    ax2.scatter(leftVc_df[leftVc_df.Community==target_name][1], leftVc_df[leftVc_df.Community==target_name][3],
                c=color, label=target_name, 
                s=cty_marker_sizes[leftVc_df.Community==target_name])
ax2.set_xlabel('2nd left-eigenvector (aka, ECI)', fontsize=16)
ax2.set_ylabel('4th left-eigenvector', fontsize=16)
#plt.legend(loc="upper center", shadow=False, scatterpoints=1, bbox_to_anchor=(0.5, -0.2), ncol=5)

ax3 = fig.add_subplot(2,3,3)
for color, i, target_name in zip(mycolors, rncomm, np.array(communities_vec)):
    ax3.scatter(leftVc_df[leftVc_df.Community==target_name][2], leftVc_df[leftVc_df.Community==target_name][3],
                c=color, label=target_name, 
                s=cty_marker_sizes[leftVc_df.Community==target_name])
ax3.set_xlabel('3rd left-eigenvector', fontsize=16)
ax3.set_ylabel('4th left-eigenvector', fontsize=16)
#plt.legend(loc="center left", shadow=False, scatterpoints=1, bbox_to_anchor=(1.1, 0.5))



#########################################################################
# SECOND ROW: RIGHT-EIGENVECTORS
ax4 = fig.add_subplot(2,3,4)
for color, i, target_name in zip(mycolors, rncomm, np.array(communities_vec)):
    ax4.scatter(-rightVc_df[rightVc_df.Community==target_name][1], -rightVc_df[rightVc_df.Community==target_name][2],
                c=color, label=target_name, 
                s=cty_marker_sizes[rightVc_df.Community==target_name])
ax4.set_xlabel('2nd right-eigenvector', fontsize=16)
ax4.set_ylabel('3rd right-eigenvector', fontsize=16)
#plt.legend(loc="best", shadow=False, scatterpoints=1)

ax5 = fig.add_subplot(2,3,5)
for color, i, target_name in zip(mycolors, rncomm, np.array(communities_vec)):
    ax5.scatter(-rightVc_df[rightVc_df.Community==target_name][1], rightVc_df[rightVc_df.Community==target_name][3],
                c=color, label=target_name, 
                s=cty_marker_sizes[rightVc_df.Community==target_name])
ax5.set_xlabel('2nd right-eigenvector', fontsize=16)
ax5.set_ylabel('4th right-eigenvector', fontsize=16)

# LEGEND
ax5legend = ax5.legend(loc="upper center", shadow=False, scatterpoints=1, bbox_to_anchor=(0.5, -0.2), ncol=6,
          title=r'$\bf{Communities}$', fontsize=16, frameon=True, fancybox=True, markerscale=1)
plt.setp(ax5legend.get_title(),fontsize=16)

ax6 = fig.add_subplot(2,3,6)
for color, i, target_name in zip(mycolors, rncomm, np.array(communities_vec)):
    ax6.scatter(-rightVc_df[rightVc_df.Community==target_name][2], rightVc_df[rightVc_df.Community==target_name][3],
                c=color, label=target_name, 
                s=cty_marker_sizes[rightVc_df.Community==target_name])
ax6.set_xlabel('3rd right-eigenvector', fontsize=16)
ax6.set_ylabel('4th right-eigenvector', fontsize=16)





plt.subplots_adjust(wspace=0.4, hspace=0.4)

#plt.axis([-2, 3, -3, 3])
plt.show()



# Principal Component Analysis

In [ ]:
ncomps = 3
pca = PCA(n_components = ncomps, whiten = True)
#X_pca = pca.fit_transform(Mc2c)
X_pca = pca.fit_transform(Mcp_widedf)

In [ ]:
fig = plt.figure(figsize=(15,5))

ax1 = fig.add_subplot(1,3,1)
for color, i, target_name in zip(mycolors, rncomm, np.array(communities_vec)):
    ax1.scatter(X_pca[leftVc_df.Community==target_name, 0], X_pca[leftVc_df.Community==target_name, 1],
                c=color, label=target_name)
ax1.set_xlabel('PC0', fontsize=16)
ax1.set_ylabel('PC1', fontsize=16)
#plt.legend(loc="best", shadow=False, scatterpoints=1)

ax2 = fig.add_subplot(1,3,2)
for color, i, target_name in zip(mycolors, rncomm, np.array(communities_vec)):
    ax2.scatter(X_pca[leftVc_df.Community==target_name, 0], X_pca[leftVc_df.Community==target_name, 2],
                c=color, label=target_name)
ax2.set_xlabel('PC0', fontsize=16)
ax2.set_ylabel('PC2', fontsize=16)
plt.legend(loc="upper center", shadow=False, scatterpoints=1, bbox_to_anchor=(0.5, -0.2), ncol=5)

ax3 = fig.add_subplot(1,3,3)
for color, i, target_name in zip(mycolors, rncomm, np.array(communities_vec)):
    ax3.scatter(X_pca[leftVc_df.Community==target_name, 1], X_pca[leftVc_df.Community==target_name, 2],
                c=color, label=target_name)
ax3.set_xlabel('PC1', fontsize=16)
ax3.set_ylabel('PC2', fontsize=16)
#plt.legend(loc="center", shadow=False, scatterpoints=1, bbox_to_anchor=(0.5, -0.05), ncol=5)


#plt.axis([-2, 3, -3, 3])
plt.show()

# Growth regressions

In [ ]:
resFA = FactorAnalysis(n_components = ncomps).fit_transform(Mcp_widedf)
resFA_df = pd.DataFrame(resFA, index=Mcp_widedf.index, columns = ["FC"+str(i) for i in range(ncomps)])

In [ ]:
resFA_df.head()

In [ ]:
dist = EComm_0001_complexities.distance_to_center(Mcp_widedf, kcomm=3)

In [ ]:
print((len(dist), len(divty)))

In [ ]:
dist.head()

In [ ]:
plt.scatter(divty, dist.rightDist2Origin)
plt.show()

In [ ]:
X_pca_df = pd.DataFrame(X_pca, index=Mcp_widedf.index, columns = ["PC" + str(i) for i in range(ncomps)])

In [ ]:
X_pca_df.head()

In [ ]:
originaleci_df = longdf[longdf['year'].astype(str).astype(int)==year][["exporter", "eci"]].drop_duplicates()
originaleci_df.index = originaleci_df["exporter"].values
originaleci_df.index.name = "exporter"

In [ ]:
originaleci_df.head()

In [ ]:
gdp2joindf = gdp_longdf[gdp_longdf["year"].astype(str).astype(int)==year]
gdp2joindf["log_gdppc"] = np.log(gdp2joindf["gdp_per_capita_constant2010USD"].values)

gdp2joindf.index = gdp2joindf["iso3c"].values

gdp2joindf.index.name = "exporter"

In [ ]:
gdp2joindf

In [ ]:
mydata = dist.join(originaleci_df[["eci"]]).join(leftVc_df[[1,2]]).join(X_pca_df).join(resFA_df).join(gdp2joindf[["log_gdppc"]], how='inner')
print(mydata.shape)
mydata.head()

In [ ]:
g = sns.pairplot(mydata, kind="reg")

In [ ]:
est = smf.ols(formula='chd ~ C(famhist)', data=df).fit()
short_summary(est)

In [ ]:
mat = np.array([[1.0,2,3],[8, 7, 2]])
mat_df = pd.DataFrame(mat, index=['c1', 'c2'], columns = ['p1', 'p2', 'p3'])
mat_df

In [ ]:
mat_df.sum().sum()*(mat_df.T/mat_df.sum(axis=1)).T/mat_df.sum(axis=0)

In [ ]:
print(mat_df.sum().sum())
print(mat_df.sum(axis=1))
print(mat_df.sum(axis=0))

In [ ]:
2.0*23.0/(17.0*5.0)

In [ ]:
np.floor(mat_df.sum().sum()*(mat_df.T/mat_df.sum(axis=1)).T/mat_df.sum(axis=0)).astype(bool).astype(int)